# Pytorch — фреймворк для обучения нейронных сетей и не только

## Зачем нужны библиотеки для обучения нейронных сетей?

* иметь качественные и эффективные реализации лоссов, слоев, оптимизаторов
* стандартизация кода, использование сторонних библиотек/моделей/имплементаций статей от коммьюнити
* ускорение с помощью GPU без написания специального кода

## Как обучаюся нейронные сети?

- методы оптимизации, использующие градиент

- для оптимизации вычисления градиента используют метод обратного распространения ошибки

Поэтому в основе любого фреймворка лежит автоматическое дифференцирование

## Фреймворки

Динамика популярности по поисковым запросам в Google:
![alt text](https://github.com/data-mining-in-action/DMIA_Industry_2019_Autumn/blob/master/seminar04/pytorch_tutorial/images/frameworks.png?raw=true)

![](https://github.com/dvpolyakov/ml_course/blob/master/images/frameworks.png?raw=1)

Tensorflow и Pytorch — два основных фреймворка на текущий момент. 

## В чём разница?

- Все фреймворки содержат приблизительно одинаковый функционал: модули для построения сетей, оптимизаторы для настройки, полезные утили, ...

- Tensorflow 1 использует статическое определение графа (сначала указанный граф компилируется, а потом через него можно прогонять данные).

- Pytorch и новый Tensorflow 2 используют динамическое определение графа — не нужно строить граф заранее.

## На практике

- Динамическое построение графа — привычная работа в python-парадигме и удобная отладка на ходу.

- На практике стоит немного понимать все популярные фреймворки — приходится изучать и использовать открытые реализации по свежим статьям.

## Pytorch

### Tensors

Тензор — многомерный массив и основной объект в Pytorch. Операции с тензорами происходят почти также, как и с массивами в numpy

In [0]:
%pylab inline
import torch

In [0]:
x = torch.Tensor()

print(x, x.type())
print("Tensor's device: ", x.device)

# x.to("cuda")  # выдаст ошибку на машине без настроенной CUDA

In [0]:
# Создание тензора "из данных"

# np.array([10., 20., 30.])

x = torch.tensor([10., 20., 30.])  
print(f"I'm {x}, my type is {x.type()}")

In [0]:
# Дополнительно мы можем указать тип тензора с помощью dtype

# np.array([10., 20., 30.], dtype=np.int32)

x = torch.tensor([10., 20., 30.], dtype=torch.long)  
print(f"I'm {x}, my type is {x.type()}")

In [0]:
# Мы можем создать тензор из np.ndarray

numpy_arr = np.eye(4)
x = torch.from_numpy(numpy_arr)
print(f"I'm {x}, my type is {x.type()}.")

In [0]:
# Метод from_numpy не создает тензор, он использает тот же участок памяти, что и массив. 
# Поэтому изменив массив -- мы изменим тензор.

numpy_arr[:, 0] = 50
print(x)

In [0]:
# Можно создавать тензоры и без данных. 

# Ниже несколько примеров, которые почти эквивалентны соответствующим в numpy
x = torch.rand(3,3) # np.random.rand(3,3)
print(f"Random tensor {x}")
x = torch.eye(3) # np.eye(3)
print(f"Identity tensor {x}")
x = torch.ones(4, 5) # np.ones((4,5))
print(f"All-ones tensor {x}")
x = torch.zeros(4, 5) # np.zeros((4,5))
print(f"All-zeros tensor {x}")

In [0]:
# Pytorch умеет превращать тензор в numpy ndarray

x = torch.rand(2, 2)
x_np = x.numpy()
print(f"I'm {x_np}, my type is {type(x_np)} ")

In [0]:
# Размер тензора можно узнать с помощью .size(), .shape

x = torch.rand(2, 2)
print("x.shape: ", x.shape, "\nx.size(): ", x.size())

In [0]:
# Менять размер тензора можно с помощью .view([s_1, s_2, s_3, ..., s_n]).
# Произведение  s_1 * ... * s_n -- должно быть равно количеству элементов.
# Одну из s_i можно заменить на -1, тогда она рассчитается автоматически

x = torch.arange(12)
print(x.view([2, 6]).numpy())
print(x.view([3, -1]).numpy()) 
print(x.view([2, 3, -1]).numpy())
try:
    print(x.view([2, 5]).numpy())
except RuntimeError as e:
    print(f"Wrong dimentions produce the following error: {e}")

In [0]:
# В заключении этого блока покажем, что изменять значение тензора можно прямым обращением по индексу.

x = torch.arange(25).view((-1, 5))
print(x)
print(x[2,4])
print(x[2])
print(x[[0,1,2,3], 0])
x[[0,1,2,3,4], 0] = 100
print(x)

**Упражнение:**

Реализуйте на PyTorch вычисление следующих функций


$$ x(t) = t - 1.5 * cos( 15 t) $$
$$ y(t) = t - 1.5 * sin( 16 t) $$

In [0]:
t = torch.linspace(-10, 10, steps = 10000)

# compute x(t) and y(t) as defined above
x = <your_code_here>
y = <your_code_here>

plt.plot(x.numpy(), y.numpy())

## Автодифференцирование

Разберёмся, как выглядит автодифференцирование на практике

In [0]:
a = torch.tensor(2.)
b = torch.tensor(1.)

c = a + b
d = b + 1
e = c * d

print(a, b, c, d, e)

Мы можем представить это в виде следующего графа:
<img src="https://github.com/data-mining-in-action/DMIA_Industry_2019_Autumn/blob/master/seminar04/pytorch_tutorial/images/tree-eval.png?raw=true" width="600">

Тепрь посчитаем частные производные по $a$ и $b$.
$$\frac{\partial e}{\partial a} = \frac{\partial e}{\partial c} \frac{\partial c}{\partial a}$$
$$\frac{\partial e}{\partial b} = \frac{\partial e}{\partial c} \frac{\partial c}{\partial b} + 
\frac{\partial e}{\partial d} \frac{\partial d}{\partial b}$$

Можно заметить, что, считая поизводные мы спускаемся про графу сверху вниз, считая производные одного узла по соседнему.

<img src="https://github.com/data-mining-in-action/DMIA_Industry_2019_Autumn/blob/master/seminar04/pytorch_tutorial/images/tree-eval-derivs.png?raw=true" width="600">

- $\frac{\partial e}{\partial a}$ равна произведению значений на ребрах по пути из $a$ в $e$.
- Из $b$ в $e$ идет два пути, и в формуле выше мы видим что мы суммируем значения, полученные для каждого из путей.

Подробнее про бэкпроп: http://colah.github.io/posts/2015-08-Backprop/

Как автоград устроен в Pytorch: https://youtu.be/MswxJw-8PvE




Чтобы посчитать градиент в Pytorch, нужно обратиться к полю .grad

In [0]:
a = torch.tensor(2., requires_grad=True)
b = torch.tensor(1., requires_grad=True)

c = a + b
d = b + 1
e = c * d
e.backward()

print("de/da = ", a.grad.item(), "de/db = ", b.grad.item())

Напишем линейную регрессию, предоставив вычисление градиента PyTorch

In [0]:
# загружаем данные
from sklearn.datasets import load_boston
boston = load_boston()
plt.scatter(boston.data[:, -1], boston.target)

In [0]:
# инизиализируем переменные, ставим флаг requires_grad=True

w = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

x = torch.tensor(boston.data[:,-1] / 10, dtype=torch.float32)
y = torch.tensor(boston.target, dtype=torch.float32)

In [0]:
# пишем, как считать предсказания и формулу лосса
y_pred = w * x + b
loss = torch.mean( (y_pred - y)**2 )

# propagete gradients
loss.backward()

In [0]:
# посчитаем градиенты по параметрам

print("dL/dw = {}\n".format(w.grad))
print("dL/db = {}\n".format(b.grad))

In [0]:
from IPython.display import clear_output

w = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

x = torch.tensor(boston.data[:,-1] / 10, dtype=torch.float32)
y = torch.tensor(boston.target, dtype=torch.float32)

for i in range(100):

    y_pred = w * x + b
    loss = torch.mean( (y_pred - y)**2 )
    loss.backward()

    w.data -= 0.05 * w.grad.data
    b.data -= 0.05 * b.grad.data
    
    #zero gradients
    w.grad.data.zero_()
    b.grad.data.zero_()
    
    # the rest of code is just bells and whistles
    if (i+1)%5==0:
        clear_output(True)
        plt.scatter(x.data.numpy(), y.data.numpy())
        plt.scatter(x.data.numpy(), y_pred.data.numpy(), color='orange', linewidth=5)
        plt.show()

        print("loss = ", loss.data.numpy())
        if loss.data.numpy() < 0.5:
            print("Done!")
            break

**Задание:** попробуйте реализовать и написать нелинейную регрессию. Попробуйте квадратичную функцию. И поставьте побольше эпох обучения.